# Find the topics about Seoul

### Topic Modeling(LDA) with KOT POI data

Which topic is the most relevant with Seoul?
Let's find it from overview text from KOT POI data about Seoul

In [1]:
import pandas as pd
import numpy as np
import gensim
import nltk
import time

from matplotlib import pyplot as plt
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from pprint import pprint
# from text_cleaning import lemmatize_and_stem, preprocess, get_aggregate_score, replace_periods

from functions import remove_html, remove_newline

In [2]:
# set database connection
from pymongo import MongoClient 
client = MongoClient('localhost', 27017)

In [3]:
# set database
db = client['cp_seoul']

In [4]:
# set collection
coll = db['seoul_poi']

In [5]:
# test db connection (get first record from db)
cursor = coll.find({})
cursor.next()

{'_id': ObjectId('5db26b80c36b1f23ca16b1d9'),
 'addr1': '38, Donggyo-ro 29-gil, Mapo-gu, Seoul',
 'areacode': '1',
 'cat1': 'A05',
 'cat2': 'A0502',
 'cat3': 'A05020900',
 'contentid': '2480979',
 'contenttypeid': '82',
 'createdtime': '20170214145634',
 'firstimage': 'http://tong.visitkorea.or.kr/cms/resource/87/2479687_image2_1.jpg',
 'firstimage2': 'http://tong.visitkorea.or.kr/cms/resource/87/2479687_image3_1.jpg',
 'mapx': '126.9230428662',
 'mapy': '37.5621620943',
 'masterid': '2479328',
 'modifiedtime': '20190125151640',
 'readcount': '3832',
 'sigungucode': '13',
 'title': '17℃ (17도씨)',
 'zipcode': '03984',
 'directions': '[Subway]<br />\nHongik Univ. Station (Seoul Subway Line 2, Gyeongui-Jungang Line, Airport Railroad), Exit 3.<br />\nWalk for approx. 5 min.<br><br>\n\n[Bus]<br />\nKolon Apt. 106-dong Entrance Bus Stop<br />\nKolon Apt. 104-dong Entrance Bus Stop<br />\nDaemyeong Apt. Bus Stop<br />\nMaeul Mapo 05',
 'dongcode': '27',
 'homepage': '<a title="open in new wind

cf)  
    82: "음식(82)Dining",
    76: "관광지(76)TouristAttractions",
    79: "쇼핑(79)Shopping",
    78: "문화시설(78)Cultural Facilities",
    80: "숙박(80)Accommodation",
    85: "축제/공연/행사(85)Festivals/Events/Performances",
    75: "레포츠(75)Leisure/Sports",
    77: "교통(77)Transportation",    

In [6]:
# load data to dataframe (only Tourist attraction and Culutral Facilities)
cursor = coll.find({'contenttypeid': {"$in": ['76', '78', ]}})
df = pd.DataFrame(list(cursor))

In [7]:
df.head().T

,0,1,2,3,4
_id,5db26b80c36b1f23ca16b1da,5db26b80c36b1f23ca16b1de,5db26b80c36b1f23ca16b1e0,5db26b80c36b1f23ca16b1e1,5db26b80c36b1f23ca16b1e4
addr1,"31, Daehak-ro 12-gil, Jongno-gu, Seoul","50, 63-ro, Yeongdeungpo-gu, Seoul","50, 63-ro, Yeongdeungpo-gu, Seoul","20-1, Samil-daero 8-gil, Jung-gu, Seoul","21, Apgujeong-ro 29-gil, Gangnam-gu, Seoul"
addr2,NaN,NaN,NaN,NaN,NaN
areacode,1,1,1,1,1
cat1,A02,A02,A02,A02,A02
cat2,A0206,A0206,A0205,A0203,A0202
cat3,A02060600,A02060500,A02050600,A02030400,A02020300
contentid,1240735,621155,264122,2590011,1295190
contenttypeid,78,78,76,76,76
createdtime,20110324102837,20080908171332,20021228150750,20190211130745,20110601095512


In [8]:
# get overview text of POIs
overviews = list(df['overview'])

In [9]:
overviews[:5]

['1m Classic Art Hall, located in Daehang-ro is a classic art experience center exclusively for children. The center provides children with the opportunity to enjoy classical music in a fun and interesting way. In the ‘1m Experience Classic’ program (the oldest children’s classical program in the nation), teachers not only provide interesting mini-music learns, but also assist children in trying out a variety of instruments.<br><br>\n\nAt the performance halls at the 1m Classic Art Hall, the stage is just 1m away from the surrounding seats, giving young audience members a more intimate musical experience.',
 '63 City in Yeouido stands 264 meters above sea level and is considered one of Seoul’s most well known landmarks. The 60th floor ‘Sky Deck’ of 63 City Building was remodeled into an art museum, offering a place for art exhibitions, magic shows, and various cultural experiences. Visitors will also be able to enjoy the beautiful scenery of Hangang River through the observatory’s larg

In [10]:
# need to clean the markup characters (<Br>, \n, ...)
overviews = list(map(remove_html, overviews))
overviews = list(map(remove_newline, overviews))

In [11]:
overviews[:5]

['1m Classic Art Hall, located in Daehang-ro is a classic art experience center exclusively for children. The center provides children with the opportunity to enjoy classical music in a fun and interesting way. In the ‘1m Experience Classic’ program (the oldest children’s classical program in the nation), teachers not only provide interesting mini-music learns, but also assist children in trying out a variety of instruments.\n\nAt the performance halls at the 1m Classic Art Hall, the stage is just 1m away from the surrounding seats, giving young audience members a more intimate musical experience.',
 '63 City in Yeouido stands 264 meters above sea level and is considered one of Seoul’s most well known landmarks. The 60th floor ‘Sky Deck’ of 63 City Building was remodeled into an art museum, offering a place for art exhibitions, magic shows, and various cultural experiences. Visitors will also be able to enjoy the beautiful scenery of Hangang River through the observatory’s large window

#### Tokenize with Lemmatization and Stemming

In [12]:
from gensim.utils import simple_preprocess

def lemmatize_and_stem(text):
    """Return Lemmatized and stemmed text"""
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos="v"))


In [13]:
# NLTK Stop words
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
#extends stopwords for this dataset
stopwords.extend([
    "korea",
    "korean",
    "dong",
    "visit",
    "include",
    "area",
    "food",
    "serve",
    "dish",
    "restaurant",
    "offer",
    "center",
    "hotel",
    "guest",
    "product"
])


In [14]:
# preprocess function
def preprocess(text):
    """Return lowercase, lemmatize, and stem text"""
    result = []

    for token in simple_preprocess(text):
        if token not in stopwords:
            result.append(lemmatize_and_stem(token))
    return result

In [15]:
# perform lemmatize_and_stem and remove stop words
overviews_token = list(map(preprocess, overviews))

In [16]:
len(overviews_token)

423

In [17]:
# Create dictionary
id2word = gensim.corpora.Dictionary(overviews_token)

In [18]:
# filtering extreme words
id2word.filter_extremes(no_below=5, no_above=0.30)

In [19]:
# make Term Document Frequency (corpus)
corpus = [id2word.doc2bow(overview) for overview in overviews_token]

In [20]:
len(corpus)

423

In [21]:
# build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus, 
                                            num_topics=5,
                                            id2word=id2word,
                                            passes=20,
                                            alpha='auto')


In [22]:
# print topics
pprint(lda_model.print_topics())

[(0,
  '0.028*"perform" + 0.026*"art" + 0.025*"hall" + 0.022*"theater" + '
  '0.015*"visitor" + 0.013*"experi" + 0.013*"exhibit" + 0.011*"program" + '
  '0.010*"music" + 0.010*"floor"'),
 (1,
  '0.043*"museum" + 0.034*"art" + 0.023*"exhibit" + 0.015*"tradit" + '
  '0.014*"galleri" + 0.010*"open" + 0.010*"shop" + 0.010*"street" + '
  '0.009*"display" + 0.009*"modern"'),
 (2,
  '0.034*"templ" + 0.013*"mountain" + 0.012*"church" + 0.012*"build" + '
  '0.012*"tower" + 0.011*"place" + 0.011*"spa" + 0.010*"year" + 0.009*"namsan" '
  '+ 0.008*"buddhist"'),
 (3,
  '0.061*"park" + 0.029*"mountain" + 0.017*"river" + 0.016*"hangang" + '
  '0.010*"tomb" + 0.010*"bridg" + 0.010*"natur" + 0.009*"forest" + '
  '0.009*"visitor" + 0.009*"fortress"'),
 (4,
  '0.033*"build" + 0.032*"palac" + 0.030*"gate" + 0.027*"king" + 0.016*"hall" '
  '+ 0.014*"joseon" + 0.013*"hous" + 0.013*"dynasti" + 0.011*"royal" + '
  '0.009*"nation"')]


*Overview text of POIs have 5 topics*

* Namsan [3]
* Park and Nature (Hangang(River)) [5]
* Museum (Traditional and Historical) [2]
* Art (Exhibition and Performs) [1]
* Interesting Area (Hongdae, Gangnam, Garosugil) [4]


#### Visualize the topics-keywords

In [23]:
# import Plotting tools (pyLDAvis)
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
# Draw plot with LDA Model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/teo/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.126694 -0.038626       1        1  27.586327
0     -0.170073  0.026908       2        1  20.694609
4      0.114900 -0.163568       3        1  19.960238
3      0.114863  0.140660       4        1  19.441275
2      0.067004  0.034626       5        1  12.317553, topic_info=    Category        Freq       Term       Total  loglift  logprob
162  Default  273.000000       park  273.000000  30.0000  30.0000
44   Default  269.000000     museum  269.000000  29.0000  29.0000
85   Default  125.000000       gate  125.000000  28.0000  28.0000
573  Default  145.000000      palac  145.000000  27.0000  27.0000
557  Default   95.000000      templ   95.000000  26.0000  26.0000
0    Default  307.000000        art  307.000000  25.0000  25.0000
19   Default  130.000000    perform  130.000000  24.0000  24.0000
66   Default  157.000000   mountain  157.000000  23.0000  23.0000
234  Default  129.000000       king  129.000000  22.0000  22.0000
459  Default   98.000000    theater   98.000000  21.0000  21.0000
36   Default  194.000000    exhibit  194.000000  20.0000  20.0000
51   Default   69.000000      river   69.000000  19.0000  19.0000
38   Default   63.000000    hangang   63.000000  18.0000  18.0000
32   Default  235.000000      build  235.000000  17.0000  17.0000
10   Default  230.000000       hall  230.000000  16.0000  16.0000
814  Default   35.000000     church   35.000000  15.0000  15.0000
567  Default   45.000000      tower   45.000000  14.0000  14.0000
187  Default   90.000000     joseon   90.000000  13.0000  13.0000
23   Default   44.000000       seat   44.000000  12.0000  12.0000
555  Default   54.000000      royal   54.000000  11.0000  11.0000
130  Default   28.000000        spa   28.000000  10.0000  10.0000
558  Default   51.000000       tomb   51.000000   9.0000   9.0000
84   Default  103.000000    galleri  103.000000   8.0000   8.0000
381  Default   74.000000       shop   74.000000   7.0000   7.0000
688  Default   42.000000     inform   42.000000   6.0000   6.0000
110  Default   41.000000     forest   41.000000   5.0000   5.0000
503  Default   49.000000      bridg   49.000000   4.0000   4.0000
180  Default   80.000000    dynasti   80.000000   3.0000   3.0000
183  Default   44.000000   fortress   44.000000   2.0000   2.0000
236  Default   49.000000   pavilion   49.000000   1.0000   1.0000
..       ...         ...        ...         ...      ...      ...
544   Topic5    7.610631       head   11.932855   1.6444  -5.8086
936   Topic5    8.073905    medicin   12.856987   1.6289  -5.7495
469   Topic5    6.726239       club   10.999087   1.6023  -5.9322
764   Topic5    9.467740         mt   15.547392   1.5981  -5.5903
67    Topic5   23.142939     namsan   42.459095   1.4873  -4.6965
846   Topic5   13.654275     martyr   25.760780   1.4593  -5.2241
133   Topic5    8.488237  treatment   14.681091   1.5463  -5.6995
498   Topic5   11.479461     valley   22.079849   1.4400  -5.3976
143   Topic5    8.432203       foot   14.667402   1.5406  -5.7061
149   Topic5   10.568798     servic   21.277325   1.3944  -5.4803
292   Topic5   13.116667      plant   29.647751   1.2786  -5.2643
48    Topic5   27.371862      place  115.557091   0.6539  -4.5287
66    Topic5   31.710150   mountain  157.180862   0.4934  -4.3815
170   Topic5   24.209257       year   98.695145   0.6888  -4.6514
72    Topic5   19.074913       view   80.491570   0.6544  -4.8898
32    Topic5   30.645203      build  235.051102   0.0568  -4.4157
403   Topic5   15.173701     garden   51.762440   0.8670  -5.1186
233   Topic5   11.749520       high   29.993599   1.1570  -5.3744
197   Topic5   16.300240       site   75.836617   0.5567  -5.0470
98    Topic5   20.363634     tradit  165.804138  -0.0029  -4.8244
156   Topic5   16.721998      first   92.319984   0.3856  -5.0214
31    Topic5   15.458566     beauti   73.017281   0.5416  -5.1000
159   Topic5   

*5 topics*

* 1 - Art (Exhibition and Performs)
* 2 - Museum (Traditional and Historical)
* 3 - Namsan
* 4 - Interesting Area (Hongdae, Gangnam, Garosugil)
* 5 - Park and Nature (Hangang(River))






# Conclusion

## LDA modeling can find meaningful topics from overview text